<a href="https://colab.research.google.com/github/tak0210/SeSAC/blob/main/(colab_GPU)parallel_stacking_ensemble_ABD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔥 지역난방 열수요 예측: 시즌별 스태킹 앙상블 + DLinear

## 📋 모델링 전략
- **시즌 분할**: Heating Season vs Non-Heating Season
- **스태킹 앙상블**: Prophet + LightGBM + GRU
- **DLinear**: 별도 실행
- **브랜치 처리**: 범주형 변수로 통합
- **총 모델 수**: 8개 (2시즌 × 4모델) + DLinear 1개

In [ ]:
# Google Colab 환경 확인 및 패키지 설치
import sys
IN_COLAB = 'google.colab' in sys.modules

# if IN_COLAB:
#     print("🔥 Google Colab 환경에서 실행 중...")
#     !pip install lightgbm prophet torch
#     from google.colab import files, drive
#     print("✅ 패키지 설치 완료!")
# else:
#     print("💻 로컬 환경에서 실행 중...")

In [2]:
# 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from tqdm.auto import tqdm

# 머신러닝
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# Prophet
try:
    from prophet import Prophet
except ImportError:
    print("Prophet 설치 필요")
    Prophet = None

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 디바이스: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()

plt.rcParams['figure.figsize'] = (12, 6)
print("📚 라이브러리 로드 완료!")

🚀 디바이스: cuda
   GPU: NVIDIA A100-SXM4-40GB
📚 라이브러리 로드 완료!


In [3]:
# 데이터 파일 로드
if IN_COLAB:
    print("📁 파일 업로드 방법 선택:")
    print("1. 직접 업로드")
    print("2. Google Drive")

    method = input("선택 (1 또는 2): ")

    if method == "1":
        uploaded = files.upload()
        files_list = list(uploaded.keys())
        train_path = [f for f in files_list if 'train' in f.lower()][0]
        test_path = [f for f in files_list if 'test' in f.lower()][0]
    else:
        drive.mount('/content/drive')
        train_path = "/content/drive/MyDrive/train_heat_ABD.csv"
        test_path = "/content/drive/MyDrive/test_heat_ABD.csv"
else:
    train_path = 'train_heat_ABD.csv'
    test_path = 'test_heat_ABD.csv'

print(f"✅ 파일 경로 설정 완료")

📁 파일 업로드 방법 선택:
1. 직접 업로드
2. Google Drive
선택 (1 또는 2): 1


Saving test_data_ABD.csv to test_data_ABD.csv
Saving train_data_ABD.csv to train_data_ABD.csv
✅ 파일 경로 설정 완료


## 1️⃣ 데이터 로드 및 전처리

In [4]:
def load_and_preprocess(train_path, test_path):
    print("📊 데이터 로드 및 전처리...")

    # 데이터 로드
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    def process_df(df):
        # 컬럼명 정리
        if 'Unnamed: 0' in df.columns:
            df = df.drop(columns=['Unnamed: 0'])
        df.columns = [col.replace('train_heat.', '') for col in df.columns]

        # 시간 변수
        df['datetime'] = pd.to_datetime(df['tm'], format='%Y%m%d%H')
        df['year'] = df['datetime'].dt.year
        df['month'] = df['datetime'].dt.month
        df['day'] = df['datetime'].dt.day
        df['hour'] = df['datetime'].dt.hour
        df['dayofweek'] = df['datetime'].dt.dayofweek

        # 결측치 처리
        missing_cols = ['ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si', 'ta_chi']
        if 'heat_demand' in df.columns:
            missing_cols.append('heat_demand')

        for col in missing_cols:
            if col in df.columns:
                df[col] = df[col].replace(-99, np.nan)

        # 일사량 야간 처리
        if 'si' in df.columns:
            night_mask = (df['hour'] < 8) | (df['hour'] > 18)
            df.loc[night_mask & df['si'].isna(), 'si'] = 0

        # 지사별 보간
        df = df.sort_values(['branch_id', 'datetime'])
        numeric_cols = df.select_dtypes(include=[np.number]).columns

        for branch in df['branch_id'].unique():
            mask = df['branch_id'] == branch
            df.loc[mask, numeric_cols] = df.loc[mask, numeric_cols].interpolate().fillna(method='ffill').fillna(method='bfill')

        return df

    train_df = process_df(train_df)
    test_df = process_df(test_df)

    print(f"   훈련: {train_df.shape}, 테스트: {test_df.shape}")
    print(f"   기간: {train_df['datetime'].min()} ~ {test_df['datetime'].max()}")

    return train_df, test_df

train_df, test_df = load_and_preprocess(train_path, test_path)

📊 데이터 로드 및 전처리...
   훈련: (52557, 17), 테스트: (26280, 17)
   기간: 2021-01-01 01:00:00 ~ 2023-12-31 23:00:00


## 2️⃣ 파생변수 생성

In [5]:
def create_features(df):
    """HDD, wind_chill, 순환형 인코딩, 범주형 변수 생성"""
    df = df.copy()

    # ⭐ HDD (수치형)
    if 'ta' in df.columns:
        df['HDD_18'] = np.maximum(18 - df['ta'], 0)
        df['HDD_20'] = np.maximum(20 - df['ta'], 0)

    # ⭐ wind_chill (수치형)
    if 'ta' in df.columns and 'ws' in df.columns:
        df['wind_chill'] = np.where(
            (df['ta'] <= 10) & (df['ws'] > 0),
            13.12 + 0.6215 * df['ta'] - 11.37 * (df['ws'] ** 0.16) + 0.3965 * df['ta'] * (df['ws'] ** 0.16),
            df['ta']
        )

    # ⭐ heating_season (범주형)
    df['heating_season'] = df['month'].isin([10, 11, 12, 1, 2, 3, 4]).astype(int)

    # 시간대 범주형
    df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
    df['is_peak_morning'] = ((df['hour'] >= 7) & (df['hour'] <= 9)).astype(int)
    df['is_peak_evening'] = ((df['hour'] >= 18) & (df['hour'] <= 22)).astype(int)
    df['is_night'] = ((df['hour'] >= 23) | (df['hour'] <= 5)).astype(int)

    # 피크시간 통합
    df['peak_time_category'] = 0
    df.loc[df['is_peak_morning'] == 1, 'peak_time_category'] = 1
    df.loc[df['is_peak_evening'] == 1, 'peak_time_category'] = 2
    df.loc[df['is_night'] == 1, 'peak_time_category'] = 3

    # ⭐ 기온 범주 (범주형)
    if 'ta' in df.columns:
        df['temp_category'] = pd.cut(df['ta'],
                                   bins=[-np.inf, 0, 10, 20, 30, np.inf],
                                   labels=[0, 1, 2, 3, 4]).astype(int)

    # ⭐ 강수 강도 (범주형)
    if 'rn_day' in df.columns:
        df['rain_intensity'] = pd.cut(df['rn_day'],
                                   bins=[-1, 0, 1, 5, 10, np.inf],
                                   labels=[0, 1, 2, 3, 4]).astype(int)

    # ⭐ 순환형 인코딩 (시간 cos, sin)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)

    return df

# 파생변수 생성
train_df = create_features(train_df)
test_df = create_features(test_df)

print(f"✅ 파생변수 생성 완료: {train_df.shape[1]}개 컬럼")

✅ 파생변수 생성 완료: 34개 컬럼


## 3️⃣ 시즌별 데이터 분할

In [6]:
# 시즌별 데이터 분할
def split_by_season(df):
    heating_data = df[df['heating_season'] == 1].copy()
    non_heating_data = df[df['heating_season'] == 0].copy()
    return heating_data, non_heating_data

train_heating, train_non_heating = split_by_season(train_df)
test_heating, test_non_heating = split_by_season(test_df)

print("📊 시즌별 데이터 분할:")
print(f"   난방시즌 - 훈련: {len(train_heating):,}, 테스트: {len(test_heating):,}")
print(f"   비난방시즌 - 훈련: {len(train_non_heating):,}, 테스트: {len(test_non_heating):,}")

📊 시즌별 데이터 분할:
   난방시즌 - 훈련: 30,525, 테스트: 15,264
   비난방시즌 - 훈련: 22,032, 테스트: 11,016


## 4️⃣ 모델 클래스 정의

In [7]:
# TimeSeriesDataset
class TimeSeriesDataset(Dataset):
    def __init__(self, data, target, sequence_length=24):
        self.data = torch.FloatTensor(data)
        self.target = torch.FloatTensor(target)
        self.sequence_length = sequence_length

    def __len__(self):
        return max(1, len(self.data) - self.sequence_length + 1)

    def __getitem__(self, idx):
        if idx >= len(self.data) - self.sequence_length:
            idx = max(0, len(self.data) - self.sequence_length)
        x = self.data[idx:idx + self.sequence_length]
        y = self.target[idx + self.sequence_length - 1]
        return x, y.unsqueeze(0)

# GRU 모델
class GRUNet(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, dropout=0.2):
        super(GRUNet, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers,
                         batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        out = out[:, -1, :]
        out = self.dropout(out)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return out

# DLinear 모델
class DLinear(nn.Module):
    def __init__(self, input_size, seq_len=24):
        super(DLinear, self).__init__()
        self.seq_len = seq_len
        self.decomposition = nn.AvgPool1d(kernel_size=25, stride=1, padding=12)
        self.Linear_Seasonal = nn.Linear(self.seq_len, 1)
        self.Linear_Trend = nn.Linear(self.seq_len, 1)
        self.feature_proj = nn.Linear(input_size, 1)

    def forward(self, x):
        x_proj = self.feature_proj(x).squeeze(-1)
        seasonal_init = self.decomposition(x_proj.unsqueeze(1)).squeeze(1)
        trend_init = x_proj - seasonal_init
        seasonal_output = self.Linear_Seasonal(seasonal_init)
        trend_output = self.Linear_Trend(trend_init)
        return (seasonal_output + trend_output).unsqueeze(-1)

print("✅ 모델 클래스 정의 완료")

✅ 모델 클래스 정의 완료


In [8]:
# Prophet 모델 클래스
class ProphetModel:
    def __init__(self):
        self.models = {}

    def fit(self, df, target_col='heat_demand'):
        if Prophet is None:
            raise ImportError("Prophet 라이브러리 필요")

        branches = df['branch_id'].unique()
        for branch in tqdm(branches, desc="Prophet 브랜치별 훈련"):
            branch_data = df[df['branch_id'] == branch].copy()

            prophet_df = pd.DataFrame({
                'ds': branch_data['datetime'],
                'y': branch_data[target_col]
            })

            model = Prophet(
                daily_seasonality=True,
                weekly_seasonality=True,
                yearly_seasonality=True,
                seasonality_mode='multiplicative'
            )

            # 회귀변수 추가
            regressors = ['hour', 'ta', 'HDD_18', 'wind_chill']
            for reg in regressors:
                if reg in branch_data.columns:
                    model.add_regressor(reg)
                    prophet_df[reg] = branch_data[reg].values

            import logging
            logging.getLogger('prophet').setLevel(logging.WARNING)
            model.fit(prophet_df)
            self.models[branch] = model

    def predict(self, df):
        predictions = []
        for branch in df['branch_id'].unique():
            if branch not in self.models:
                predictions.extend([0] * len(df[df['branch_id'] == branch]))
                continue

            branch_data = df[df['branch_id'] == branch].copy()
            future_df = pd.DataFrame({'ds': branch_data['datetime']})

            regressors = ['hour', 'ta', 'HDD_18', 'wind_chill']
            for reg in regressors:
                if reg in branch_data.columns:
                    future_df[reg] = branch_data[reg].values

            forecast = self.models[branch].predict(future_df)
            predictions.extend(forecast['yhat'].values)

        return np.array(predictions)

# LightGBM 모델 클래스 (브랜치 원핫인코딩)
class LightGBMModel:
    def __init__(self):
        self.model = None
        self.feature_cols = None

    def fit(self, df, target_col='heat_demand'):
        exclude_cols = ['tm', 'datetime', 'year', target_col]

        # 브랜치 원핫인코딩
        df_encoded = pd.get_dummies(df, columns=['branch_id'], prefix='branch')

        self.feature_cols = [col for col in df_encoded.columns if col not in exclude_cols]
        X = df_encoded[self.feature_cols]
        y = df_encoded[target_col]

        device_type = 'gpu' if torch.cuda.is_available() else 'cpu'

        self.model = lgb.LGBMRegressor(
            device=device_type,
            n_estimators=1000,
            learning_rate=0.05,
            max_depth=8,
            num_leaves=31,
            random_state=42,
            n_jobs=-1
        )

        self.model.fit(X, y)

    def predict(self, df):
        df_encoded = pd.get_dummies(df, columns=['branch_id'], prefix='branch')

        # 컬럼 맞추기
        for col in self.feature_cols:
            if col not in df_encoded.columns:
                df_encoded[col] = 0

        X = df_encoded[self.feature_cols]
        return self.model.predict(X)

# GRU 모델 클래스 (브랜치 임베딩)
class GRUModel:
    def __init__(self, sequence_length=24):
        self.model = None
        self.scaler = MinMaxScaler()
        self.sequence_length = sequence_length
        self.device = device
        self.feature_cols = None
        self.branch_encoder = None

    def fit(self, df, target_col='heat_demand'):
        exclude_cols = ['tm', 'datetime', 'year', target_col, 'branch_id']
        self.feature_cols = [col for col in df.columns
                           if col not in exclude_cols and df[col].dtype in ['int64', 'float64']]

        # 브랜치 인코딩
        self.branch_encoder = LabelEncoder()
        branch_encoded = self.branch_encoder.fit_transform(df['branch_id'])

        X = df[self.feature_cols].values
        y = df[target_col].values

        X = np.nan_to_num(X, nan=0)
        y = np.nan_to_num(y, nan=0)

        X_scaled = self.scaler.fit_transform(X)

        dataset = TimeSeriesDataset(X_scaled, y, self.sequence_length)
        dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

        if len(dataset) == 0:
            return

        self.model = GRUNet(X_scaled.shape[1]).to(self.device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=0.001)

        self.model.train()
        for epoch in tqdm(range(30), desc="GRU 훈련"):
            for batch_x, batch_y in dataloader:
                batch_x = batch_x.to(self.device)
                batch_y = batch_y.to(self.device)

                optimizer.zero_grad()
                outputs = self.model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

    def predict(self, df):
        if self.model is None:
            return np.full(len(df), 0)

        X = df[self.feature_cols].values
        X = np.nan_to_num(X, nan=0)
        X_scaled = self.scaler.transform(X)

        self.model.eval()
        predictions = []

        with torch.no_grad():
            for i in range(len(X_scaled)):
                if i < self.sequence_length:
                    predictions.append(0)
                else:
                    seq_data = X_scaled[i-self.sequence_length+1:i+1]
                    seq_tensor = torch.FloatTensor(seq_data).unsqueeze(0).to(self.device)
                    pred = self.model(seq_tensor).cpu().numpy()[0, 0]
                    predictions.append(pred)

        return np.array(predictions)

print("✅ 개별 모델 클래스 정의 완료")

✅ 개별 모델 클래스 정의 완료


## 5️⃣ 스태킹 앙상블 클래스

In [9]:
class StackingEnsemble:
    def __init__(self):
        self.models = {
            'prophet': ProphetModel(),
            'lightgbm': LightGBMModel(),
            'gru': GRUModel()
        }
        self.meta_model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.individual_scores = {}

    def fit(self, train_df, target_col='heat_demand'):
        print("🚀 스태킹 앙상블 훈련 시작...")

        # 검증용 데이터 분할
        val_size = int(len(train_df) * 0.2)
        val_df = train_df.iloc[-val_size:].copy()
        train_fit_df = train_df.iloc[:-val_size].copy()

        level1_predictions = {}

        # 각 모델 훈련 및 예측
        for name, model in self.models.items():
            print(f"\n📊 {name.upper()} 훈련 중...")
            try:
                start_time = datetime.now()
                model.fit(train_fit_df, target_col)
                train_time = (datetime.now() - start_time).total_seconds()

                val_pred = model.predict(val_df)
                level1_predictions[name] = val_pred

                # 개별 모델 성능 계산
                rmse = np.sqrt(mean_squared_error(val_df[target_col], val_pred))
                mae = mean_absolute_error(val_df[target_col], val_pred)

                self.individual_scores[name] = {'rmse': rmse, 'mae': mae}

                print(f"   📈 {name} 성능: RMSE={rmse:.4f}, MAE={mae:.4f}")
                print(f"   ⏱️ 훈련 시간: {train_time:.1f}초")

            except Exception as e:
                print(f"   ❌ {name} 훈련 실패: {e}")
                level1_predictions[name] = np.full(len(val_df), val_df[target_col].mean())
                self.individual_scores[name] = {'rmse': 999, 'mae': 999}

        # 메타 모델 훈련
        print(f"\n🎯 메타 모델 훈련...")
        meta_features = np.column_stack(list(level1_predictions.values()))
        self.meta_model.fit(meta_features, val_df[target_col])

        # 스태킹 성능
        stacking_pred = self.meta_model.predict(meta_features)
        stacking_rmse = np.sqrt(mean_squared_error(val_df[target_col], stacking_pred))
        stacking_mae = mean_absolute_error(val_df[target_col], stacking_pred)

        self.individual_scores['stacking'] = {'rmse': stacking_rmse, 'mae': stacking_mae}
        print(f"   📈 스태킹 성능: RMSE={stacking_rmse:.4f}, MAE={stacking_mae:.4f}")
        print("✅ 스태킹 앙상블 훈련 완료")

    def predict(self, test_df):
        level1_predictions = {}

        for name, model in self.models.items():
            try:
                level1_predictions[name] = model.predict(test_df)
            except Exception as e:
                print(f"❌ {name} 예측 실패: {e}")
                level1_predictions[name] = np.full(len(test_df), 0)

        meta_features = np.column_stack(list(level1_predictions.values()))
        final_pred = self.meta_model.predict(meta_features)

        return final_pred, level1_predictions

print("✅ 스태킹 앙상블 클래스 정의 완료")

✅ 스태킹 앙상블 클래스 정의 완료


## 6️⃣ DLinear 모델 클래스

In [10]:
class DLinearModel:
    def __init__(self, sequence_length=24):
        self.model = None
        self.scaler = MinMaxScaler()
        self.sequence_length = sequence_length
        self.device = device
        self.feature_cols = None

    def fit(self, df, target_col='heat_demand'):
        print("📊 DLinear 훈련 중...")

        exclude_cols = ['tm', 'datetime', 'year', target_col, 'branch_id']
        self.feature_cols = [col for col in df.columns
                           if col not in exclude_cols and df[col].dtype in ['int64', 'float64']]

        X = df[self.feature_cols].values
        y = df[target_col].values

        X = np.nan_to_num(X, nan=0)
        y = np.nan_to_num(y, nan=0)

        X_scaled = self.scaler.fit_transform(X)

        dataset = TimeSeriesDataset(X_scaled, y, self.sequence_length)
        dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

        if len(dataset) == 0:
            print("⚠️ 데이터셋이 비어있음")
            return

        self.model = DLinear(X_scaled.shape[1], self.sequence_length).to(self.device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=0.001)

        self.model.train()
        for epoch in tqdm(range(20), desc="DLinear 훈련"):
            total_loss = 0
            for batch_x, batch_y in dataloader:
                batch_x = batch_x.to(self.device)
                batch_y = batch_y.to(self.device)

                optimizer.zero_grad()
                outputs = self.model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

        print(f"✅ DLinear 훈련 완료")

    def predict(self, df):
        if self.model is None:
            return np.full(len(df), 0)

        X = df[self.feature_cols].values
        X = np.nan_to_num(X, nan=0)
        X_scaled = self.scaler.transform(X)

        self.model.eval()
        predictions = []

        with torch.no_grad():
            for i in range(len(X_scaled)):
                if i < self.sequence_length:
                    predictions.append(0)
                else:
                    seq_data = X_scaled[i-self.sequence_length+1:i+1]
                    seq_tensor = torch.FloatTensor(seq_data).unsqueeze(0).to(self.device)
                    pred = self.model(seq_tensor).cpu().numpy()[0, 0, 0]
                    predictions.append(pred)

        return np.array(predictions)

    def evaluate(self, df, target_col='heat_demand'):
        predictions = self.predict(df)

        if target_col in df.columns:
            y_true = df[target_col].values
            rmse = np.sqrt(mean_squared_error(y_true, predictions))
            mae = mean_absolute_error(y_true, predictions)
            return {'rmse': rmse, 'mae': mae}
        else:
            return {'rmse': None, 'mae': None}

print("✅ DLinear 모델 클래스 정의 완료")

✅ DLinear 모델 클래스 정의 완료


## 7️⃣ 모델 훈련 및 평가

In [11]:
# 시즌별 스태킹 앙상블 훈련
print("🔥 시즌별 스태킹 앙상블 훈련 시작!")
print("=" * 60)

# 난방시즌 모델
print("\n❄️ 난방시즌 모델 훈련")
heating_ensemble = StackingEnsemble()
heating_ensemble.fit(train_heating)

# 비난방시즌 모델
print("\n🌞 비난방시즌 모델 훈련")
non_heating_ensemble = StackingEnsemble()
non_heating_ensemble.fit(train_non_heating)

print("\n✅ 시즌별 스태킹 앙상블 훈련 완료!")

🔥 시즌별 스태킹 앙상블 훈련 시작!

❄️ 난방시즌 모델 훈련
🚀 스태킹 앙상블 훈련 시작...

📊 PROPHET 훈련 중...


Prophet 브랜치별 훈련:   0%|          | 0/3 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmpnbacjblg/zq5_3fku.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnbacjblg/mnjiz293.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=66548', 'data', 'file=/tmp/tmpnbacjblg/zq5_3fku.json', 'init=/tmp/tmpnbacjblg/mnjiz293.json', 'output', 'file=/tmp/tmpnbacjblg/prophet_modelj_38qj4d/prophet_model-20250610015657.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:56:57 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:57:05 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnbacjblg/pa_1dkx8.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnbacjblg/14uf0_4n.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

   📈 prophet 성능: RMSE=2824.5588, MAE=1903.8641
   ⏱️ 훈련 시간: 25.8초

📊 LIGHTGBM 훈련 중...
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2672
[LightGBM] [Info] Number of data points in the train set: 24420, number of used features: 31
[LightGBM] [Info] Using GPU Device: NVIDIA A100-SXM4-40GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 22 dense feature groups (0.56 MB) transferred to GPU in 0.002021 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 248.868182
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

GRU 훈련:   0%|          | 0/30 [00:00<?, ?it/s]

...

Prophet 브랜치별 훈련:   0%|          | 0/3 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmpnbacjblg/kt8pdbec.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnbacjblg/zvupzuza.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=70651', 'data', 'file=/tmp/tmpnbacjblg/kt8pdbec.json', 'init=/tmp/tmpnbacjblg/zvupzuza.json', 'output', 'file=/tmp/tmpnbacjblg/prophet_modelwsd7u06w/prophet_model-20250610015823.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:58:23 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:58:26 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnbacjblg/y5p3_zan.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnbacjblg/eign4q8i.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

   📈 prophet 성능: RMSE=89.4820, MAE=79.5461
   ⏱️ 훈련 시간: 11.4초

📊 LIGHTGBM 훈련 중...
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2422
[LightGBM] [Info] Number of data points in the train set: 17626, number of used features: 31
[LightGBM] [Info] Using GPU Device: NVIDIA A100-SXM4-40GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 20 dense feature groups (0.34 MB) transferred to GPU in 0.001783 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 56.360745
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits 

GRU 훈련:   0%|          | 0/30 [00:00<?, ?it/s]

   📈 gru 성능: RMSE=26.2578, MAE=19.7200
   ⏱️ 훈련 시간: 29.3초

🎯 메타 모델 훈련...
   📈 스태킹 성능: RMSE=5.3038, MAE=3.9931
✅ 스태킹 앙상블 훈련 완료

✅ 시즌별 스태킹 앙상블 훈련 완료!


In [12]:
# DLinear 모델 별도 훈련
print("\n🚀 DLinear 모델 별도 훈련")
print("-" * 40)

dlinear_model = DLinearModel()
dlinear_model.fit(train_df)

print("✅ DLinear 모델 훈련 완료!")


🚀 DLinear 모델 별도 훈련
----------------------------------------
📊 DLinear 훈련 중...


DLinear 훈련:   0%|          | 0/20 [00:00<?, ?it/s]

✅ DLinear 훈련 완료
✅ DLinear 모델 훈련 완료!


## 8️⃣ 테스트 예측 및 성능 평가

In [13]:
# 시즌별 예측
print("🎯 테스트 데이터 예측 시작...")

# 난방시즌 예측
if len(test_heating) > 0:
    heating_pred, heating_individual = heating_ensemble.predict(test_heating)
    print(f"❄️ 난방시즌 예측: {len(heating_pred):,}개")
else:
    heating_pred, heating_individual = np.array([]), {}
    print("❄️ 난방시즌 테스트 데이터 없음")

# 비난방시즌 예측
if len(test_non_heating) > 0:
    non_heating_pred, non_heating_individual = non_heating_ensemble.predict(test_non_heating)
    print(f"🌞 비난방시즌 예측: {len(non_heating_pred):,}개")
else:
    non_heating_pred, non_heating_individual = np.array([]), {}
    print("🌞 비난방시즌 테스트 데이터 없음")

# DLinear 예측
dlinear_pred = dlinear_model.predict(test_df)
print(f"🔬 DLinear 예측: {len(dlinear_pred):,}개")

print("✅ 모든 예측 완료!")

🎯 테스트 데이터 예측 시작...
❄️ 난방시즌 예측: 15,264개
🌞 비난방시즌 예측: 11,016개
🔬 DLinear 예측: 26,280개
✅ 모든 예측 완료!


In [14]:
# 성능 결과 정리
print("\n📊 모델별 성능 요약")
print("=" * 60)

print("\n❄️ 난방시즌 모델 성능:")
for model_name, scores in heating_ensemble.individual_scores.items():
    print(f"   {model_name:12s}: RMSE={scores['rmse']:.4f}, MAE={scores['mae']:.4f}")

print("\n🌞 비난방시즌 모델 성능:")
for model_name, scores in non_heating_ensemble.individual_scores.items():
    print(f"   {model_name:12s}: RMSE={scores['rmse']:.4f}, MAE={scores['mae']:.4f}")

# DLinear 성능 (타겟이 있는 경우)
if 'heat_demand' in test_df.columns:
    dlinear_scores = dlinear_model.evaluate(test_df)
    print(f"\n🔬 DLinear 모델 성능:")
    print(f"   dlinear     : RMSE={dlinear_scores['rmse']:.4f}, MAE={dlinear_scores['mae']:.4f}")
else:
    print(f"\n🔬 DLinear 모델: 테스트 타겟 없음 (예측만 수행)")


📊 모델별 성능 요약

❄️ 난방시즌 모델 성능:
   prophet     : RMSE=2824.5588, MAE=1903.8641
   lightgbm    : RMSE=59.9356, MAE=46.7466
   gru         : RMSE=87.3477, MAE=70.4077
   stacking    : RMSE=10.8629, MAE=8.0997

🌞 비난방시즌 모델 성능:
   prophet     : RMSE=89.4820, MAE=79.5461
   lightgbm    : RMSE=15.2900, MAE=11.7409
   gru         : RMSE=26.2578, MAE=19.7200
   stacking    : RMSE=5.3038, MAE=3.9931

🔬 DLinear 모델 성능:
   dlinear     : RMSE=142.5999, MAE=117.2322


## 9️⃣ 최종 예측 결과 저장

In [ ]:
# 최종 예측 결과 통합
print("💾 최종 예측 결과 저장...")

# 기본 데이터프레임 생성
result_df = test_df[['tm', 'branch_id']].copy()

# 시즌별 예측 결과 통합
final_stacking_pred = np.zeros(len(test_df))

if len(test_heating) > 0:
    heating_mask = test_df['heating_season'] == 1
    final_stacking_pred[heating_mask] = heating_pred

if len(test_non_heating) > 0:
    non_heating_mask = test_df['heating_season'] == 0
    final_stacking_pred[non_heating_mask] = non_heating_pred

# 결과 저장
result_df['stacking_prediction'] = np.maximum(final_stacking_pred, 0)
result_df['dlinear_prediction'] = np.maximum(dlinear_pred, 0)

# 파일 저장
result_df.to_csv('heat_demand_predictions_season_stacking.csv', index=False)
print(f"📁 예측 결과 저장: heat_demand_predictions_season_stacking.csv")

# 통계 출력
print(f"\n📈 예측값 통계:")
print(f"   스태킹 - 평균: {result_df['stacking_prediction'].mean():.2f}, 표준편차: {result_df['stacking_prediction'].std():.2f}")
print(f"   DLinear - 평균: {result_df['dlinear_prediction'].mean():.2f}, 표준편차: {result_df['dlinear_prediction'].std():.2f}")

# 지사별 통계
branch_stats = result_df.groupby('branch_id')[['stacking_prediction', 'dlinear_prediction']].agg(['mean', 'std'])
print(f"\n📊 지사별 예측 통계:")
print(branch_stats.round(2))

# Google Drive 저장 (Colab)
if IN_COLAB:
    save_drive = input("\nGoogle Drive에 저장? (y/n): ").lower() == 'y'
    if save_drive:
        try:
            !cp heat_demand_predictions_season_stacking.csv /content/drive/MyDrive/
            print("✅ Google Drive 저장 완료!")
        except:
            print("⚠️ Google Drive 저장 실패")

print("\n🎊 모든 작업 완료!")

## 🎯 최종 요약

In [ ]:
print("\n📋 🔥 최종 분석 요약 🔥")
print("=" * 60)

print(f"✅ 모델 구성:")
print(f"   📊 시즌별 스태킹 앙상블 (Prophet + LightGBM + GRU)")
print(f"   🔬 DLinear 모델 (별도 실행)")
print(f"   📋 총 모델 수: 8개 (2시즌 × 4모델) + DLinear 1개")

print(f"\n📈 파생변수:")
print(f"   ⭐ HDD, wind_chill (수치형)")
print(f"   🔄 순환형 인코딩 (시간 cos, sin)")
print(f"   📋 범주형: branch_id, heating_season, 피크시간, 기온범주, 강수강도")

print(f"\n🏆 시즌별 접근법:")
print(f"   ❄️ 난방시즌 (10,11,12,1,2,3,4월): 전용 모델")
print(f"   🌞 비난방시즌 (5,6,7,8,9월): 전용 모델")
print(f"   🎯 브랜치 처리: Prophet(회귀변수), LightGBM(원핫), GRU(임베딩)")

print(f"\n📁 출력 파일:")
print(f"   • heat_demand_predictions_season_stacking.csv")
print(f"   • 컬럼: tm, branch_id, stacking_prediction, dlinear_prediction")

# 최고 성능 모델 찾기
best_models = []
for season, ensemble in [('난방', heating_ensemble), ('비난방', non_heating_ensemble)]:
    if ensemble.individual_scores:
        best_model = min(ensemble.individual_scores.items(), key=lambda x: x[1]['rmse'])
        best_models.append((season, best_model[0], best_model[1]['rmse']))

if best_models:
    print(f"\n🥇 시즌별 최고 성능:")
    for season, model, rmse in best_models:
        print(f"   {season}시즌: {model.upper()} (RMSE: {rmse:.4f})")

print(f"\n🎉 지역난방 열수요 예측 완료!")
print(f"🔬 혁신적 접근: 시즌별 모델 분할 + 브랜치 범주형 처리")

# GPU 메모리 정리
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"🧹 GPU 메모리 정리 완료")